# TARPES analysis

## note
As the real tarpes data is really larlge (making it impractical to include this package), so here we create a mock Xarray resembling the data and use it for explanation.

The mockl data is a list that contains xarray:

* 20 x 20 matrix data
* Angle (-10deg to 10deg) and energy (5 eV to 6eV)
* 1000 "position" attributes, for the delay line positilon (100.0mm to 150.00 mm), which is converted to delay time.
*  

In [ ]:
import numpy as np
import xarray as xr





